2020-01-24: Let's test if we get numba to work in the analysis routines.
We are using channels heavily and numba doesn't know about them. Let's teach ti

https://numba.pydata.org/numba-doc/latest/extending/interval-example.html

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("/global/homes/r/rkube/repos/delta")
import numpy as np
import json

In [3]:
import numba

Start with the channel class. This class boils down to a class with 3 integers ch_v, ch_h and ch_num, and 1 string, dev

In [4]:
# taken from analysis/channels.py
def ch_num_to_vh(ch_num):
    """Returns a tuple (ch_v, ch_h) for a channel number. Note that these are 1-based
    numbers."""
    assert((ch_num >= 1) & (ch_num < 193))
    # Calculate using zero-base
    ch_num -= 1
    ch_v = ch_num // 8
    ch_h = ch_num - ch_v * 8
    return(ch_v + 1, ch_h + 1)


def ch_vh_to_num(ch_v, ch_h, debug=True):
    """Returns the channel number 1..192 from a channel h and v"""

    # We usually want to check that we are within the bounds.
    # But sometimes it is helpful to avoid this.
    if debug:
        assert((ch_v > 0) & (ch_v < 25))
        assert((ch_h > 0) & (ch_h < 9))

    return((ch_v - 1) * 8 + ch_h)


class channel():
    """Represents an ECEI channel.
    The ECEI array has 24 horizontal channels and 8 vertical channels.

    They are commonly represented as
    L2203
    where L denotes ???, 22 is the horizontal channel and 08 is the vertical channel.
    """

    def __init__(self, dev, ch_v, ch_h):
        """
        Input:
        ======
        dev: string, must be in 'L' 'H' 'G' 'GT' 'GR' 'HR'
        ch_h: int, Horizontal channel number, between 1 and 24
        ch_v: int, Vertical channel number, between 1 and 8
        """

        assert(dev in ['L', 'H', 'G', 'HT', 'GR', 'HR'])
        # 24 horizontal channels
        assert((ch_v > 0) & (ch_v  < 25))
        # 8 vertical channels
        assert((ch_h > 0) & (ch_h < 9))

        self.ch_v = ch_v
        self.ch_h = ch_h
        self.dev = dev
        self.ch_num = ch_vh_to_num(self.ch_v, self.ch_h)

    @classmethod
    def from_str(cls, ch_str):
        """Generates a channel object from a string, such as L2204 or GT1606.
        Input:
        ======
        cls: The class object (this is never passed to the method, but akin to self)
        ch_str: A channel string, such as L2205 of GT0808
        """

        import re
        # Define a regular expression that matches a sequence of 1 to 2 characters in [A-Z]
        # This will be our dev.
        m = re.search('[A-Z]{1,2}', ch_str)
        try:
            dev = m.group(0)
        except:
            raise AttributeError("Could not parse channel string {0:s}".format(ch_str))

        # Define a regular expression that matches 4 consecutive digits
        # These will be used to calculate ch_h and ch_v
        m = re.search('[0-9]{4}', ch_str)
        ch_num = int(m.group(0))
        
        ch_h = (ch_num % 100)
        ch_v = int(ch_num // 100)

        channel1 = cls(dev, ch_v, ch_h)
        return(channel1)



    def __str__(self):
        """Prints the channel as a standardized string DDHHVV, where D is dev, H is ch_h and V is ch_v.
        DD can be 1 or 2 characters, H and V are zero-padded"""
        ch_str = "{0:s}{1:02d}{2:02d}".format(self.dev, self.ch_v, self.ch_h)

        return(ch_str)


    def __eq__(self, other):
        """Define equality for two channels when all three, dev, ch_h, and ch_v are equal to one another."""
        return (self.dev, self.ch_v, self.ch_h) == (other.dev, other.ch_v, other.ch_h)

    def idx(self):
        """Returns the linear, ZERO-BASED, index corresponding to ch_h and ch_v"""
        return ch_vh_to_num(self.ch_v, self.ch_h) - 1

    def to_json(self):
        """Returns the class in JSON notation. 
           This method avoids serialization error when using non-standard int types,
           such as np.int64 etc..."""
        d = {"ch_v": int(self.ch_v), "ch_h": int(self.ch_h), "dev": self.dev, "ch_num": int(self.ch_num)}
        return json.dumps(d, default=lambda o: o.__dict__, sort_keys=True, indent=4)

    @classmethod
    def from_json(cls, str):
        """Returns a channel instance from a json string"""

        j = json.loads(str)

        channel1 = cls(j["dev"], int(j["ch_v"]), int(j["ch_h"]))
        return(channel1)

According to the guide, we need to create a numba type that represents instances of the channel.

In [5]:
from numba import types

class channel_nmb(types.Type):
    def __init__(self):
        super(channel_nmb, self).__init__(name='channel')

ch_numba = channel_nmb()

We now tell numba that an instance of the channel class should be treated as a  channel_nmb

In [6]:
from numba.extending import typeof_impl

@typeof_impl.register(channel)
def typeof_index(val, c):
    return ch_numba

We now have to teach numba how to construct channel types from numba functions

In [7]:
from numba.extending import type_callable

@type_callable(channel)
def type_channel(context):
    def typer(dev, ch_v, ch_h):
        if isinstance(dev, types.CharSeq) and isinstance(ch_v, types.Integer) and isinstance(ch_h, types.Integer):
            return ch_numba
    return typer

Let's define the data model for the channel class. We use an immutable struct. ch_v and ch_h are int8 and
dev is unicode_type:
https://stackoverflow.com/questions/56463147/how-to-specify-the-string-data-type-when-using-numba

In [8]:
from numba.extending import models, register_model

@register_model(channel_nmb)
class channel_model(models.StructModel):
    def __init__(self, dmm, fe_type):
        members = [
            ('dev', types.unicode_type),
            ('ch_v', types.int64),
            ('ch_h', types.int64),
            ]
        models.StructModel.__init__(self, dmm, fe_type, members)

We now want to implement the 3-argument constructor

In [9]:
from numba.extending import lower_builtin
from numba import cgutils

@lower_builtin(channel, types.CharSeq, types.Integer, types.Integer)
def impl_channel(context, builder, sig, args):
    typ = sig.return_type
    dev, ch_v, ch_h = args
    channel = cgutils.create_struct_proxy(typ)(context, builder)
    channel.dev = dev
    channel.ch_v = ch_v
    channel.ch_h = ch_h
    return channel._getvalue()

Implement boxing and unboxing:

In [10]:
from numba.extending import unbox, NativeValue

@unbox(channel_nmb)
def unbox_channel(typ, obj, c):
    """
    Convert a Interval object to a native interval structure.
    """
    print(type(c), dir(c))
    dev_obj = c.pyapi.object_getattr_string(obj, "dev")
    chv_obj = c.pyapi.object_getattr_string(obj, "ch_v")
    chh_obj = c.pyapi.object_getattr_string(obj, "ch_h")
    channel = cgutils.create_struct_proxy(typ)(c.context, c.builder)
    print(channel, type(channel), dir(channel))

    channel.dev = c.pyapi.string_as_string(dev_obj)
    channel.ch_v = c.pyapi.float_as_double(ch_v_obj)
    channel.ch_h = c.pyapi.float_as_double(ch_h_obj)
    c.pyapi.decref(dev_obj)
    c.pyapi.decref(chv_obj)
    c.pyapi.decref(chh_obj)
    is_error = cgutils.is_not_null(c.builder, c.pyapi.err_occurred())
    return NativeValue(channel._getvalue(), is_error=is_error)

In [11]:
from numba.extending import box

@box(channel_nmb)
def box_channel(typ, val, c):
    """
    Convert a native interval structure to an Interval object.
    """
    channel = cgutils.create_struct_proxy(typ)(c.context, c.builder, value=val)
    print(channel, type(channel), dir(channel))
    dev_obj = c.pyapi.string_as_string(channel.dev)
    chh_obj = c.pyapi.float_from_double(channel.ch_h)
    chv_obj = c.pyapi.float_from_double(channel.ch_v)
    class_obj = c.pyapi.unserialize(c.pyapi.serialize_object(channel))
    res = c.pyapi.call_function_objargs(class_obj, (dev_obj, chv_obj, chh_obj))
    c.pyapi.decref(dev_obj)
    c.pyapi.decref(chv_obj)
    c.pyapi.decref(chh_obj)
    c.pyapi.decref(class_obj)
    return res

In [12]:
from numba import jit

In [13]:
@jit(nopython=True)
def test_channel(ch, h):
    ch
    #return ch.ch_h < h

In [14]:
c = channel('L', 14, 5)

test_channel(c, 10)

<class 'numba.pythonapi._UnboxContext'> ['__add__', '__class__', '__contains__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getnewargs__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rmul__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '_asdict', '_field_defaults', '_fields', '_fields_defaults', '_make', '_replace', 'builder', 'context', 'count', 'index', 'pyapi', 'unbox']


TypeError: Failed in nopython mode pipeline (step: nopython mode backend)
Invalid store of i8* to {i8*, i64, i32, i32, i64, i8*, i8*} in <__main__.channel_model object at 0x2aaad39c9490> (trying to write member #0)

In [13]:
with open("../configs/test_skw.json") as df:
    cfg = json.load(df)

In [19]:
chrg_ref = channel_range.from_str(cfg["task_list"][0]["ref_channels"])
chrg_cmp = channel_range.from_str(cfg["task_list"][0]["cmp_channels"])

chpair_list = [channel_pair(ch1, ch2) for ch1, ch2 in zip(chrg_ref, chrg_cmp)]

In [24]:
@numba.jit(nopython=True,cache=True)
def cross_phase(fft_data, ch_it, fft_config, info_dict):
    """Kernel that calculates the cross-phase between two channels.
    Input:
    ======
    fft_data: ndarray, float: Contains the fourier-transformed data. 
                dim0: channel, dim1: Fourier Coefficients, dim2: STFT (bins in fluctana code)
    ch_it: iterable, Iterator over a list of channels we wish to perform our computation on

    Returns:
    ========
    Axy: float, the cross phase
    """    
    c1_idx = np.array([ch_pair.ch1.idx() for ch_pair in ch_it])
    c2_idx = np.array([ch_pair.ch2.idx() for ch_pair in ch_it])
    Pxy = (fft_data[c1_idx, :, :] * fft_data[c2_idx, :, :].conj()).mean(axis=2)
    return(np.arctan2(Pxy.real, Pxy.imag).real, info_dict)

In [25]:
cross_phase(fft_data, chpair_list, fft_params,)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
non-precise type pyobject
[1] During: typing of argument at <ipython-input-24-ee0942e72d44> (14)

File "<ipython-input-24-ee0942e72d44>", line 14:
def cross_phase(fft_data, ch_it, fft_config, info_dict):
    <source elided>
    """    
    c1_idx = np.array([ch_pair.ch1.idx() for ch_pair in ch_it])
    ^

This error may have been caused by the following argument(s):
- argument 1: Cannot type list element of <class 'analysis.channels.channel_pair'>
- argument 2: cannot determine Numba type of <class 'dict'>

This is not usually a problem with Numba itself but instead often caused by
the use of unsupported features or an issue in resolving types.

To see Python/NumPy features supported by the latest release of Numba visit:
http://numba.pydata.org/numba-doc/latest/reference/pysupported.html
and
http://numba.pydata.org/numba-doc/latest/reference/numpysupported.html

For more information about typing errors and how to d

In [52]:
GG.dtype

dtype('complex128')